## Running One Subject

In [ ]:
import os  # interact with the filesystem
from subprocess import Popen, PIPE, STDOUT  # enable calling commandline
import matplotlib.pyplot as plt  # manipulate figures
import seaborn as sns  # display results
import pandas as pd   # manipulate tabular 
import warnings
import time
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=ResourceWarning) 

## Setup Data

Set the "working directory" path. 

In [ ]:
data_dir = '/projects/niblab/bids_projects/Experiments/Bevel/testing_beta' #RENCI PATH
#data_dir = '/Users/nikkibytes/Documents/niblunc/testing_beta' #LOCAL PATH
print('Our working directory: {}'.format(data_dir))

Set our `*.tsv` events file


We take the original `*events.tsv` file and extract the columns we need to run the betaseries analysis.
For this we simply take 4 specific columns and then fill them into the `sub-*_task_prob_run-*_events.tsv` originally generated as output from BIDS. 

Here you can see we load the original `*.tsv` file, rename column names from the original to the standarize name, "outcome becomes "trial_type" and "RT" becomes "response_time". 

## If we need to format our `*.tsv` files:

In [ ]:
events_file = os.path.join(data_dir, 'sub-001/func/sub-001_task-prob_run-1_events.tsv')
events_df = pd.read_csv(events_file, sep="\t", na_values="n/a")
events_df.head()

In [ ]:
# Rename our original columns
events_df.rename({"outcome": "trial_type"}, axis='columns', inplace=True)
events_df.head()
events_df.rename({"RT": "response_time"}, axis='columns', inplace=True)
events_df.head()

In [ ]:
events_df = events_df.dropna(axis="rows")
events_df.head()

In [ ]:
new_df= events_df[["onset", "duration", 'trial_type', 'response_time']]
new_df.head()

In [ ]:
newfile = os.path.join(data_dir, 'sub-001/func/sub-001_task-prob_run-1_events.tsv')
new_df.to_csv(newfile, sep="\t", na_rep="n/a", index=False)

## Setup atlas file

In [ ]:
atlas_txt = os.path.join(data_dir,
                         "derivatives",
                         "data",
                         "Schaefer2018_100Parcels_7Networks_order.txt")
atlas_df = pd.read_csv(atlas_txt, sep="\t", header=None)
atlas_df.head()

atlas_df.drop([2, 3, 4, 5], axis='columns', inplace=True)
atlas_df.head()

atlas_df.rename({0: 'index', 1: 'regions'}, axis='columns', inplace=True)
atlas_df.head()

atlas_df.replace(regex={'7Networks_(.*)': '\\1'}, inplace=True)
atlas_df.head()

atlas_tsv = atlas_txt.replace(".txt", ".tsv")
atlas_df.to_csv(atlas_tsv, sep="\t", index=False)

## Set the nibs command and run
Here we set our flags for betaseries, print to check it and then run  
Note 
    * the `-c` flag is our confound flag, "the confound column names that are to be included in nuisance regression. write the confounds you wish to include separated by a space". <WhiteMatter, CSF, Cosine0, X, RotX, etc.....>

    * `-sp` which is our space derivative, where we select a bold derivative in a specific space to be used, currenlty only space available is the `MNI152NLin2009cAsym`
    * -`sm` is our smoothing kernel (optional)
    * the other inputs are specific directories

In [ ]:
out_dir = os.path.join(data_dir, "derivatives/basic_output")
work_dir = os.path.join(out_dir, "work_dir_test")
atlas_mni_file = os.path.join(data_dir,
                              "derivatives",
                              "data",
                              "Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz")
print('Our output directory: {}'.format(out_dir))
print('Our working output directory: {}'.format(work_dir))

In [ ]:
# Run parallel
# 
cmd = """\
nibs \
{bids_dir} \
fmriprep \
{out_dir} \
participant \
-sm 6 \
--run_label 4 \
-c WhiteMatter CSF \
-sp MNI152NLin2009cAsym \
-w {work_dir} \
-a {atlas_mni_file} \
-l {atlas_tsv} \
""".format(atlas_mni_file=atlas_mni_file,
           atlas_tsv=atlas_tsv,
           bids_dir=os.path.join(data_dir),
           out_dir=out_dir,
           work_dir=work_dir)





# Since we cannot run bash commands inside this tutorial
# we are printing the actual bash command so you can see it
# in the output
print("The Example Command:\n", cmd)



In [ ]:
# call nibs

p = Popen(cmd, shell=True, stdout=PIPE, stderr=STDOUT)

while True:
    line = p.stdout.readline()
    if not line:
        break
    print(line)
    

In [ ]:
corr_mat_path = os.path.join(out_dir, "NiBetaSeries", "nibetaseries", "sub-001", "func")
trial_types = ['punish','reward']
filename_template = "sub-001_task-prob_run-1_bold_space-MNI152NLin2009cAsym_preproc_trialtype-{trial_type}_matrix.tsv"
pd_dict = {}

for trial_type in trial_types:
    file_path = os.path.join(corr_mat_path, filename_template.format(trial_type=trial_type))
    print(file_path)
    pd_dict[trial_type] = pd.read_csv(file_path, sep='\t', na_values="n/a", index_col=0)
# display example matrix
    print(pd_dict[trial_type].head())


In [ ]:


fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True, figsize=(10, 30),
                         gridspec_kw={'wspace': 0.025, 'hspace': 0.075})

cbar_ax = fig.add_axes([.91, .3, .03, .4])
r = 0

for trial_type, df in pd_dict.items():
    g = sns.heatmap(df, ax=axes[r], center=0, square=True,
                    cbar=True, cbar_ax=cbar_ax)
    axes[r].set_title(trial_type)
    # iterate over rows
    r += 1
plt.tight_layout()
plt.savefig(os.path.join(corr_mat_path, "run-1_task_heatmap.png"))
